In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
import time
import pandas as pd
from datetime import datetime, timedelta

from netunicorn.client.remote import RemoteClient, RemoteClientException
from netunicorn.base import Experiment, ExperimentStatus, Pipeline, DockerImage
from netunicorn.library.tasks.capture.tcpdump import StartCapture, StopNamedCapture
from netunicorn.library.tasks.upload.fileio import UploadToFileIO
from netunicorn.library.tasks.upload.webdav import UploadToWebDav
from netunicorn.library.tasks.basic import SleepTask
from netunicorn.library.tasks.measurements.ookla_speedtest import SpeedTest
from netunicorn.library.tasks.video_watchers.youtube_watcher import WatchYouTubeVideo
from netunicorn.library.tasks.video_watchers.vimeo_watcher import WatchVimeoVideo
from netunicorn.library.tasks.video_watchers.twitch_watcher import WatchTwitchStream

In [3]:
netunicorn_login = 'cs190n4'
netunicorn_password = 'zvL89JkW'

NETUNICORN_ENDPOINT = os.environ.get('NETUNICORN_ENDPOINT', 'https://pinot.cs.ucsb.edu/netunicorn')
NETUNICORN_LOGIN = os.environ.get('NETUNICORN_LOGIN', netunicorn_login)
NETUNICORN_PASSWORD = os.environ.get('NETUNICORN_PASSWORD', netunicorn_password)

client = RemoteClient(endpoint=NETUNICORN_ENDPOINT, login=NETUNICORN_LOGIN, password=NETUNICORN_PASSWORD)
print("Health Check: {}".format(client.healthcheck()))
nodes = client.get_nodes()
print(nodes)

Health Check: True
[<Uncountable node pool with next node template: [aws-fargate-A-cs190n4-, aws-fargate-B-cs190n4-, aws-fargate-ARM64-cs190n4-]>]


In [4]:
pipeline = (
    Pipeline()
    .then(StartCapture(filepath="/tmp/youtube_capture.pcap", name="youtube_capture"))
    .then(WatchYouTubeVideo("https://www.youtube.com/watch?v=dQw4w9WgXcQ", 10))
    .then(StopNamedCapture(start_capture_task_name="youtube_capture"))
    .then(StartCapture(filepath="/tmp/vimeo_capture.pcap", name="vimeo_capture"))
    .then(WatchVimeoVideo("https://vimeo.com/375468729", 10))
    .then(StopNamedCapture(start_capture_task_name="vimeo_capture"))
    .then(StartCapture(filepath="/tmp/twitch_capture.pcap", name="twitch_capture"))
    .then(WatchTwitchStream("https://www.twitch.tv/shroud", 10))
    .then(StopNamedCapture(start_capture_task_name="twitch_capture"))
    .then(UploadToWebDav(
        filepaths={
            "/tmp/youtube_capture.pcap",
            "/tmp/vimeo_capture.pcap",
            "/tmp/twitch_capture.pcap"
        },
        endpoint="http://snl-server-5.cs.ucsb.edu/cs190n/cs190n4/capture",
        username="uploader",
        password="uploader"
    ))
)

In [5]:
working_node = "aws"

In [6]:
def execute_pipeline(pipeline, working_node, experiment_label):
    working_nodes = nodes.filter(lambda node: node.name.startswith(working_node)).take(1)
    experiment = Experiment().map(pipeline, working_nodes)

    for deployment in experiment:
        # you can explore the image on the DockerHub
        deployment.environment_definition = DockerImage(image="satyandraguthula/netunicorn_images")

    try:
        client.delete_experiment(experiment_label)
    except RemoteClientException:
        pass

    # Prepare Experiment
    client.prepare_experiment(experiment, experiment_label)
    while True:
        info = client.get_experiment_status(experiment_label)
        print(info.status)
        if info.status == ExperimentStatus.READY:
            break
        time.sleep(20)

    time.sleep(5)

    # Execute Experiment
    client.start_execution(experiment_label)
    while True:
        info = client.get_experiment_status(experiment_label)
        print(info.status)
        if info.status != ExperimentStatus.RUNNING:
            break
        time.sleep(20)
    return info

In [7]:
experiment_label = "team-4-experiment"

In [8]:
execute_pipeline(pipeline, working_node, experiment_label)

ExperimentStatus.PREPARING
ExperimentStatus.READY
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.RUNNING
ExperimentStatus.FINISHED


ExperimentExecutionInformation:
status: ExperimentStatus.FINISHED
experiment: 
 - Deployment: Node=aws-fargate-A-cs190n4-1, executor_id=9e2db76b-b617-44af-88f3-59380ba6fb11, prepared=True, error=None
execution_result:
[DeploymentExecutionResult:
  Node: aws-fargate-A-cs190n4-1
  Result: <class 'returns.result.Failure'>
   <Failure: defaultdict(<class 'list'>, {'youtube_capture': [<Success: 8>], 'dd71f943-5131-4926-8347-d9325745bed0': [<Failure: Traceback (most recent call last):
  File "/usr/local/lib/python3.10/site-packages/netunicorn/executor/utils.py", line 32, in decorator
    result = function(*args, **kwargs)
  File "/home/tigeryu/.virtualenvs/CMPSC 190N/lib/python3.10/site-packages/netunicorn/library/tasks/video_watchers/youtube_watcher.py", line 157, in run
  File "/home/tigeryu/.virtualenvs/CMPSC 190N/lib/python3.10/site-packages/netunicorn/library/tasks/video_watchers/youtube_watcher.py", line 73, in watch
  File "/usr/local/lib/python3.10/site-packages/selenium/webdriver/re